In [5]:
# 参考https://github.com/langchain-ai/langgraph/blob/main/docs/docs/tutorials/plan-and-execute/plan-and-execute.ipynb


# 使用 python-dotenv 加载 .env 文件
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.messages import HumanMessage, SystemMessage
from langchain_tavily import TavilySearch
import os

# 加载当前目录下的 .env 文件
load_dotenv(override=True)

# 获取环境变量
api_key = os.getenv("API_KEY")
base_url = os.getenv("BASE_URL")
tavily_key = os.getenv("TAVILY_API_KEY")
model_name = os.getenv("MODEL_NAME")
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"


print("API_KEY=", api_key)
print("BASE_URL=", base_url)
print("MODEL_NAME=", model_name)
print("tavily_key=", tavily_key)

# 初始化模型
model = ChatOpenAI(
    base_url=base_url,
    model=model_name,
    api_key=api_key,
    temperature=1,
)

user_query = "现任澳大利亚网球公开赛冠军的家乡是哪里?"

API_KEY= sk-a1d5476da3ea4a169ab5c10e2b424a25
BASE_URL= https://dashscope.aliyuncs.com/compatible-mode/v1
MODEL_NAME= qwen3-max
tavily_key= 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


In [6]:
# plan 节点
from pydantic import BaseModel, Field
from typing import List


class Plan(BaseModel):
    """执行计划"""

    steps: List[str] = Field(description="要遵循的不同步骤，应按顺序排列")


plan_sys_prompt = """针对给定的目标，制定一个简单的分步计划。 
该计划应包含各项具体任务，若执行得当，将得出正确答案。请勿添加任何多余步骤。 
最后一步的结果应该是最终答案。确保每一步都包含了所有必要的信息——不要跳过任何步骤。

【输出格式要求】
    输出必须严格遵循以下JSON结构：
    {{
        "steps": [
            "步骤1",
            "步骤2",
            "...",
            "最后一步"
        ]
    }}
现在，请根据用户的查询生成执行计划，并以 JSON 格式返回。
"""


planner = model.with_structured_output(Plan)

response = planner.invoke(
    [
        SystemMessage(content=plan_sys_prompt),
        HumanMessage(content=user_query),
    ],
    
)

# for chunk in stream:
#     print(chunk, end="")

print("计划：", response)

计划： steps=['确定现任澳大利亚网球公开赛男单和女单冠军分别是谁（以最近一届比赛为准）。', '查找男单冠军的家乡。', '查找女单冠军的家乡。', '根据问题意图（未指定性别），默认提供最新一届男女单打冠军各自的家乡信息作为答案。']


In [ ]:

# plan execute 节点
# 初始化 Tavily 搜索工具
tavily_search = TavilySearch(
    max_results=5,
    topic="general",
)

# 天气工具
from langchain.tools import tool


@tool
def get_weather(city: str) -> str:
    """获取指定城市的天气信息
    Args:
        city (str): 城市名称
    """
    # 这里可以集成实际的天气API调用
    return f"{city}的天气是晴朗，温度25摄氏度。"


from langchain.agents import create_agent

react_agent = create_agent(
    tools=[tavily_search, get_weather],
    model=model,
    # system_prompt="You are a helpful research assistant. Use web search to find accurate, up-to-date information.",
)

# user_query = "去网上查一下2025年英雄联盟S15总决赛在哪里举办？"


for token, metadata in react_agent.stream(
    {"messages": [{"role": "user", "content": user_query}]},
    stream_mode="messages",
):
    # print(token)
    # print("\n")
    content = token.content
    if content:
        print(content, end="")
    # elif chunk.tool_calls:
    #     print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")

我来帮您查询这两个问题。关于天气信息我可以通过工具获取，但英雄联盟S15总决赛的信息需要搜索来确认。上海的天气是晴朗，温度25摄氏度。{"query": "2025年英雄联盟S15总决赛 位置 主力场地", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.sohu.com/a/917592239_122464149", "title": "2025英雄联盟全球总决赛（S15）：赛程公布、场馆安排 - 搜狐", "content": "2025英雄联盟全球总决赛(S15)将于10月14日至11月9日在中国北京、上海、成都三地举行，这是继2017年和2020年后，该项顶级电竞赛事第三次在中国举办。赛事总", "score": 0.86767644, "raw_content": null}, {"url": "https://ol.3dmgame.com/gl/312865.html", "title": "LOLS15全球总决赛举办地点一览_3DM网游", "content": "* 主站 * 商城 * 论坛 * 自运营 登录;) 注册;) * 首页 * 单机 * 手游 * 软件 * 网游 * 自运营 * 商城 * 新闻 * 电竞 * 评测 * 视频 * 攻略 您的位置：网游 >  *图文攻略*  > LOLS15全球总决赛举办地点一览 # LOLS15全球总决赛举办地点一览 ### LOLS15全球总决赛举办地点一览 入围赛与瑞士轮阶段 – 北京智慧电竞赛事中心 淘汰赛阶段 – 上海梅赛德斯-奔驰文化中心 冠亚军决赛 – 成都东安湖体育公园多功能体育馆 更多2025全球总决赛相关内容，敬请关注微博@英雄联盟赛事，赛事官网 lpl.qq.com LOLS15全球总决赛举办地点一览 注册;) * 魔兽世界时光服 公测 暂无 09-23 * 天龙八部·归来 公测 领取 07-25 * 天龙八部·归来 限号测试 暂无 04-18 * 诛仙世界 同行测试 暂无 12-29 * 《LOL》2025全球总决赛11月9日T1 VS KT比赛视频 * 《LOL》2025全球总决赛11月2日T1 VS TES比赛视频 * 《LOL》2025全球总决赛1